# Hyperparameters

In [ ]:
# training data paramters
MAX_ROWS_PARAMETER = 100
TRAINING_PERCENT_PARAMETER = 0.3

# Hyperparameters
PARAM_K_NEAREST_NEIGHBORS = 2
PARAM_DECISION_TREE_MAX_DEPTH = 5

# PARAM_SVM_C = 1
# PARAM_SVM_GAMMA = 0.1
# PARAM_SVM_KERNEL = 'rbf'

# Development Parameters
DEBUG_PARAMETER = True
PARAM_GENERATE_CONFIDENCE_MATRIX = False

# Initializing Data

## Preparing Data

In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config
from sklearn.utils import shuffle

from sklearn import tree

import pandas as pd
import numpy as np
import seaborn as sns
import time
from matplotlib import pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

# HalvingGridSearchCV is experimental and the API might change without any deprecation cycle
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

ModuleNotFoundError: No module named 'seaborn'

### Load the Data

In [ ]:
plays_path = '/Users/davidstange/Development/github/davestange/data/big_data_bowl_2025/plays.csv'
plays = pd.read_csv(plays_path)

plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,...,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone
1,2022091809,3698,(2:13) (Shotgun) J.Burrow pass short right to ...,4,1,10,CIN,DAL,CIN,8,...,4,0.002847,-0.002847,-0.240509,True,NaN,NaN,0,Quarters,Zone
2,2022103004,3146,(2:00) (Shotgun) D.Mills pass short right to D...,4,3,12,HOU,TEN,HOU,20,...,6,0.000205,-0.000205,-0.218480,True,NaN,NaN,0,Quarters,Zone
3,2022110610,348,(9:28) (Shotgun) P.Mahomes pass short left to ...,1,2,10,KC,TEN,TEN,23,...,4,-0.001308,0.001308,-0.427749,True,NaN,NaN,0,Quarters,Zone
4,2022102700,2799,(2:16) (Shotgun) L.Jackson up the middle to TB...,3,2,8,BAL,TB,TB,27,...,-1,0.027141,-0.027141,-0.638912,False,MAN,READ OPTION,0,Cover-1,Man
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16119,2022110604,1051,(12:49) (Shotgun) T.Lawrence pass incomplete s...,2,3,4,JAX,LV,JAX,31,...,0,-0.024741,0.024741,-1.391687,True,NaN,NaN,0,Cover-2,Zone
16120,2022103005,3492,(12:32) (Shotgun) K.Cousins pass incomplete de...,4,1,10,MIN,ARI,MIN,25,...,0,-0.026580,0.026580,-0.503208,True,NaN,NaN,0,Cover-3,Zone
16121,2022092502,2337,(9:59) (Shotgun) P.Mahomes scrambles right end...,3,1,10,KC,IND,IND,13,...,10,-0.013790,0.013790,1.073898,True,NaN,NaN,0,Quarters,Zone
16122,2022091809,719,(:45) C.Rush pass incomplete deep right to C.L...,1,1,10,DAL,CIN,CIN,47,...,0,-0.011561,0.011561,-0.522397,True,UNDEFINED,NaN,0,Cover-3,Zone


## Readying the dataset

In [ ]:
def get_position_features(s: pd.Series) -> pd.Series:
    ### determine wr, te, rb based on alignment; WRxTE
    wr_count = te_count = rb_count = 0
    extra_ol = s.offenseFormation == "JUMBO"
    if not isinstance(s.receiverAlignment, float):
        alignment = s.receiverAlignment.split('x')
        if len(alignment):
            wr_count = int(alignment[0])
            te_count = int(alignment[1])
            rb_count = 5 - wr_count - te_count
            if extra_ol and rb_count > 1:
                rb_count -= 1 

    return pd.Series([wr_count, te_count, rb_count, extra_ol], ['wr_count', 'te_count', 'rb_count', 'extra_ol'])
def get_time_features(s: pd.Series) -> pd.Series:
    ### determine total time remaining, is overtime, or within two mins 
    two_min_1st_half = 32 * 60
    two_min_2st_half = 2 * 60
    two_min_overtime = 68 * 60
    
    quarter = int(s.quarter)
    secs_remaining = math.nan
    is_inside_two_mins = is_overtime = False
    quarter_clock = s.gameClock.split(':')
    if len(quarter_clock) == 2:
        secs_in_quarter = int(quarter_clock[0]) * 60 + int(quarter_clock[1])
        if quarter <= 4:
            secs_remaining = (4 - quarter) * 15 * 60 + secs_in_quarter
        else:
            secs_remaining = secs_in_quarter
            is_overtime = True
        is_inside_two_mins = secs_in_quarter <= 120 and (quarter == 2 or quarter >= 4)

    return pd.Series([secs_remaining, is_inside_two_mins, is_overtime], ['secs_remaining', 'is_inside_two_mins', 'is_overtime'])

# v1 result - just sneak, kneel, spikie, pass, or run
plays['play_result'] = plays.apply(lambda p: 
                                   'qb_sneak' if p.qbSneak == True else 
                                   'qb_kneel' if p.qbKneel == True else 
                                   'qb_spike' if p.qbSpike == True else 
                                   'pass' if p.isDropback == True else 
                                   'run', axis = 1)
plays

# get position counts (TE, WR, etc)
tmp = plays.apply(get_position_features, axis=1)
plays['wr_count'] = tmp['wr_count']
plays['te_count'] = tmp['te_count']
plays['rb_count'] = tmp['rb_count']
plays['extra_ol'] = tmp['extra_ol']

tmp = plays.apply(get_time_features, axis=1)
plays['secs_remaining'] = tmp['secs_remaining']
plays['is_inside_two_mins'] = tmp['is_inside_two_mins']
plays['is_overtime'] = tmp['is_overtime']

# # preSnapHomeScore, preSnapVisitorScore, need Home/Away 
# plays['off_score_lead'] = plays.apply(...)

# # preSnapHomeTeamWinProbability	preSnapVisitorTeamWinProbability, need Home/Away 
# plays['off_win_probability'] = plays.apply(...)

plays['off_yards_to_goal'] = plays.apply(lambda p: 100 - int(p.yardlineNumber) if p.yardlineSide == p.possessionTeam else int(p.yardlineNumber), axis = 1)

## Prepare the training set

In [ ]:
TARGET_FEATURE = 'play_result'
TRAINING_FEATURES = ['down', 'yardsToGo',                                                       # down and distance
                     'quarter', 'secs_remaining', 'is_inside_two_mins', 'is_overtime',          # game time
                     'wr_count', 'te_count', 'rb_count', 'extra_ol', 'offenseFormation',        # formations
                     TARGET_FEATURE]

# set transformer of remaining categorical features
col_trans = make_column_transformer(
    (OneHotEncoder(drop = 'if_binary'), ['offenseFormation']),
    remainder='passthrough')

In [ ]:
tmp = plays[TRAINING_FEATURES]
X_train, X_test, y_train, y_test = train_test_split(tmp.drop(TARGET_FEATURE, axis = 1), 
                                                    tmp[TARGET_FEATURE],
                                                    train_size=TRAINING_PERCENT_PARAMETER,
                                                    random_state=42)
if DEBUG_PARAMETER:
    print(f"The training set has {len(y_train)} rows")
    print(f"The test set has {len(y_test)} rows")
    print(f"The target feature is {TARGET_FEATURE} and the {len(X_train.columns)} training columns are {[x for x in X_train.columns]}")

class ModelTrainer:
    def __init__(self, model_name, actual_model):
        self.start_time = time.time()
        self.model_name = model_name
        self.model = actual_model
        
    def elapsed(self):
        elapsed_time = time.time() - self.start_time
        return f"{elapsed_time:.4f} secs"

    def run(self):
        training_features = 0 if self.model_name == "Dummy" else len(TRAINING_FEATURES) - 1
        self.model.fit(X_train, y_train)
        train_accuracy = accuracy_score(self.model.predict(X_train), y_train)
        predictions = self.model.predict(X_test)
        test_accuracy = accuracy_score(predictions, y_test) 
        print(f"| {self.model_name} | {training_features} | {self.elapsed()} | {train_accuracy:4f} | {test_accuracy:4f} |")

        if PARAM_GENERATE_CONFIDENCE_MATRIX:
            cm = confusion_matrix(predictions, y_test)
            sns.heatmap(cm, annot=True, fmt = "d", cmap = "Blues", annot_kws={"size": 20}, cbar = False)
            plt.ylabel('Actual')
            plt.xlabel('Predicted')
            plt.title(f'Confusion matrix for {self.model_name}')
            sns.set(font_scale = 2)


The training set has 4837 rows
The test set has 11287 rows
The target feature is play_result and the 11 training columns are ['down', 'yardsToGo', 'quarter', 'secs_remaining', 'is_inside_two_mins', 'is_overtime', 'wr_count', 'te_count', 'rb_count', 'extra_ol', 'offenseFormation']


# Validating & Debugging Features

In [ ]:
tv_watching = ['gameId', 'playId', 'playDescription', 'quarter', 'gameClock', 'down', 'yardsToGo', 'off_yards_to_goal', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'preSnapHomeScore', 'preSnapVisitorScore', 'secs_remaining', 'is_inside_two_mins', 'is_overtime']

steelers_bengals = plays[plays.gameId == 2022091103].sort_values(by='playId')

steelers_bengals[tv_watching].head(20)
# steelers_bengals.head(20)




# plays[plays.pff_manZone != 'Zone'][plays.pff_manZone != 'Man'][plays.pff_manZone != 'Other'][['pff_manZone', 'playDescription']]

,gameId,playId,playDescription,quarter,gameClock,down,yardsToGo,off_yards_to_goal,absoluteYardlineNumber,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,secs_remaining,is_inside_two_mins,is_overtime
10263,2022091103,58,(15:00) (Shotgun) M.Trubisky pass short middle...,1,15:00,1,10,75,35,PIT,CIN,PIT,25,0,0,3600,False,False
1278,2022091103,82,"(14:39) (No Huddle, Shotgun) M.Trubisky pass s...",1,14:39,2,7,72,38,PIT,CIN,PIT,28,0,0,3579,False,False
2595,2022091103,111,(14:21) (Shotgun) M.Trubisky pass short left t...,1,14:21,3,2,67,43,PIT,CIN,PIT,33,0,0,3561,False,False
6254,2022091103,176,(13:31) (Shotgun) J.Burrow sacked at CIN 17 fo...,1,13:31,1,10,78,88,CIN,PIT,CIN,22,0,0,3511,False,False
9518,2022091103,195,(12:52) (Shotgun) J.Burrow pass short right in...,1,12:52,2,15,83,93,CIN,PIT,CIN,17,0,0,3472,False,False
6579,2022091103,253,(12:43) (Shotgun) J.Mixon left guard to CIN 30...,1,12:43,1,10,75,85,CIN,PIT,CIN,25,0,7,3463,False,False
5868,2022091103,274,(12:05) (No Huddle) J.Mixon left tackle to CIN...,1,12:05,2,5,70,80,CIN,PIT,CIN,30,0,7,3425,False,False
459,2022091103,295,"(11:34) (No Huddle, Shotgun) J.Burrow pass sho...",1,11:34,1,10,64,74,CIN,PIT,CIN,36,0,7,3394,False,False
7468,2022091103,319,(10:55) (No Huddle) J.Mixon left end to CIN 39...,1,10:55,2,7,61,71,CIN,PIT,CIN,39,0,7,3355,False,False
1398,2022091103,340,(10:13) (Shotgun) J.Burrow pass short left to ...,1,10:13,3,7,61,71,CIN,PIT,CIN,39,0,7,3313,False,False


# Modeling

## Determine Hyperparameters

### K-Nearest Neighbors

In [ ]:
pipeline = Pipeline([
    ('transformer', col_trans),
    ('regression', KNeighborsClassifier())
])
parameters_to_try = {'regression__n_neighbors': np.array(range(1, 20))}

from sklearn.model_selection import GridSearchCV
model_finder = GridSearchCV(estimator = pipeline, 
                            param_grid = parameters_to_try, 
                            scoring = "accuracy",
                            cv = 5)
model_finder.fit(X_train, y_train)
model_finder.best_estimator_

# PARAM_K_NEAREST_NEIGHBORS = 2

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['offenseFormation'])])),
                ('regression', KNeighborsClassifier(n_neighbors=2))])

### Decision Tree

In [ ]:
pipeline = Pipeline([
    ('transformer', col_trans),
    ('regression', DecisionTreeClassifier(criterion='entropy'))
])
parameters_to_try = {'regression__max_depth': np.array(range(1, 20))}

from sklearn.model_selection import GridSearchCV
model_finder = GridSearchCV(estimator = pipeline, 
                            param_grid = parameters_to_try, 
                            scoring = "accuracy",
                            cv = 5)
model_finder.fit(X_train, y_train)
model_finder.best_estimator_

# PARAM_DECISION_TREE_MAX_DEPTH = 5

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['offenseFormation'])])),
                ('regression',
                 DecisionTreeClassifier(criterion='entropy', max_depth=5))])

### Support Vector Machines

In [ ]:
pipeline = Pipeline([
    ('transformer', col_trans),
    ('regression', SVC())
])
parameters_to_try = {
    'regression__C': [0.1, 1, 10, 100, 1000], 
	'regression__gamma': [0.1, 1, 10, 100], # 1, 0.1, 0.01, 0.001, 0.0001
	'regression__kernel': ['rbf', 'poly', 'linear']
}

# scoring on HalvingGridSearchCV is failing due to wild
# UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan.
# model_finder = HalvingGridSearchCV(estimator = pipeline, 
#                                    param_grid = parameters_to_try, 
#                                    scoring = "accuracy",
#                                    verbose=1)

# switching to 
model_finder = GridSearchCV(estimator = pipeline, 
                            param_grid = parameters_to_try, 
                            scoring = "accuracy",
                            cv = 5,
                            verbose=1)
model_finder.fit(X_train, y_train)
model_finder.best_estimator_

# PARAM_SVM_C = 1
# PARAM_SVM_GAMMA = 0.1
# PARAM_SVM_KERNEL = 'rbf'

Fitting 5 folds for each of 60 candidates, totalling 300 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


## Runtime Models

### Baseline

In [12]:
modeller = ModelTrainer(
    "Dummy", 
    DummyClassifier()
)
modeller.run()

| Dummy | 0 | 0.0121 secs | 0.599545 | 0.603615 |


### Logistic Regression

In [17]:
modeller = ModelTrainer(
    "LogisticRegression", 
    Pipeline([
        ('transformer', col_trans),
        ('regression', LogisticRegression(solver='liblinear', random_state=42, fit_intercept=True))
    ])
)
modeller.run()

| LogisticRegression | 11 | 0.0655 secs | 0.724623 | 0.736600 |


### K-Nearest Neighbors

In [21]:
modeller = ModelTrainer(
    f"KNN with {PARAM_K_NEAREST_NEIGHBORS} neighbors", 
    Pipeline([
        ('transformer', col_trans),
        ('regression', KNeighborsClassifier(n_neighbors=PARAM_K_NEAREST_NEIGHBORS))
    ])
)
modeller.run()

| KNN with 2 neighbors | 11 | 0.2155 secs | 0.778582 | 0.601311 |


### Decision Tree

In [24]:
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=PARAM_DECISION_TREE_MAX_DEPTH)

modeller = ModelTrainer(
    f"DecisionTreeClassifier with max_depth={PARAM_DECISION_TREE_MAX_DEPTH}", 
    Pipeline([
        ('transformer', col_trans),
        ('regression', decision_tree)
    ])
)
modeller.run()

| DecisionTreeClassifier with max_depth=5 | 11 | 0.0491 secs | 0.726897 | 0.738283 |


In [ ]:
# get cleaned up feature names
feature_names = []
feature_names += TRAINING_FEATURES
feature_names += col_trans.get_feature_names_out().tolist()
feature_names

feature_names = [x.replace('ordinalencoder__', '_').replace('onehotencoder__', '_').replace('remainder__', '_') for x in feature_names]

fig = plt.figure(figsize=(10,10))
_ = tree.plot_tree(decision_tree,
                   feature_names=feature_names,
                   filled=True)
plt.suptitle("Decision Tree Classifier")

## Support Vector Machines

In [ ]:
svc = SVC() #kernel=PARAM_SVM_KERNEL, gamma=PARAM_SVM_GAMMA, C=PARAM_SVM_C)

modeller = ModelTrainer(
    f"SVM (default params)",
    # f"SVM (kernel={PARAM_SVM_KERNEL}, gamma={PARAM_SVM_GAMMA}, C={PARAM_SVM_C})", 
    Pipeline([
        ('transformer', col_trans),
        ('regression', svc)
    ])
)
modeller.run()

| SVM (default params) | 11 | 2.1202 secs | 0.599545 | 0.603615 |


# Finalizing

## Model Performance

| Model | Features | Train Time | Train Accuracy | Test Accuracy |
| ----- | -------- | ---------- | -------------  | -----------   |
| Dummy | 0 | 0.0121 secs | 0.599545 | 0.603615 |
| LogisticRegression | 11 | 0.0655 secs | 0.724623 | 0.736600 |
| KNN with 2 neighbors | 11 | 0.2155 secs | 0.778582 | 0.601311 |
| DecisionTreeClassifier with max_depth=5 | 11 | 0.0491 secs | 0.726897 | 0.738283 |
| SVM (default params) | 11 | 2.1202 secs | 0.599545 | 0.603615 |